    000_16_230114_141_subway_transer_07
    
    data : 020_경의중앙선_주소데이터_20221122.csv
     save : subway_info_11.csv
     display : subway_info_11.html

In [153]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [154]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [155]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [156]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/020_경의중앙선_주소데이터_20221122.csv ',encoding='cp949')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소
0,코레일,경의중앙,가좌,서울특별시 서대문구 남가좌동 296-12,서울시 서대문구 수색로 27
1,코레일,경의중앙,강매,경기도 고양시 덕양구 행신동 1115-1,"경기도 고양시 덕양구 소원로202 (행신동,강매역사)"
2,코레일,경의중앙,곡산,경기도 고양시 일산동구 백석동 1185-1,경기도 고양시 일산동구 경의로 160
3,코레일,경의중앙,공덕,서울특별시 마포구 도화동 25-13,서울특별시 마포구 마포대로 92
4,코레일,경의중앙,구리,경기도 구리시 인창동 244-1,경기도 구리시 건원대로 34번길 32-29


In [157]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  58 non-null     object
 1   선명       58 non-null     object
 2   역명       58 non-null     object
 3   지번주소     58 non-null     object
 4   도로명주소    58 non-null     object
dtypes: object(5)
memory usage: 2.4+ KB


In [158]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [159]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [160]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["지번주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["지번주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


서울특별시 성동구 응봉동 246은(는) 검색에 실패했습니다


,철도운영기관명,선명,역명,지번주소,도로명주소,x,y
0,코레일,경의중앙,가좌,서울특별시 서대문구 남가좌동 296-12,서울시 서대문구 수색로 27,948355.073243349,1952323.10746635
1,코레일,경의중앙,강매,경기도 고양시 덕양구 행신동 1115-1,"경기도 고양시 덕양구 소원로202 (행신동,강매역사)",942197.984765547,1957230.09587581
2,코레일,경의중앙,곡산,경기도 고양시 일산동구 백석동 1185-1,경기도 고양시 일산동구 경의로 160,938321.795367442,1961059.15305838
3,코레일,경의중앙,공덕,서울특별시 마포구 도화동 25-13,서울특별시 마포구 마포대로 92,951517.685545804,1949441.02788465
4,코레일,경의중앙,구리,경기도 구리시 인창동 244-1,경기도 구리시 건원대로 34번길 32-29,968816.570408409,1956190.17460319


In [161]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [162]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,코레일,경의중앙,가좌,서울특별시 서대문구 남가좌동 296-12,서울시 서대문구 수색로 27,948355.073243349,1952323.10746635,126.915181,37.568829
1,코레일,경의중앙,강매,경기도 고양시 덕양구 행신동 1115-1,"경기도 고양시 덕양구 소원로202 (행신동,강매역사)",942197.984765547,1957230.09587581,126.845076,37.612689
2,코레일,경의중앙,곡산,경기도 고양시 일산동구 백석동 1185-1,경기도 고양시 일산동구 경의로 160,938321.795367442,1961059.15305838,126.800836,37.646948
3,코레일,경의중앙,공덕,서울특별시 마포구 도화동 25-13,서울특별시 마포구 마포대로 92,951517.685545804,1949441.02788465,126.951183,37.543024
4,코레일,경의중앙,구리,경기도 구리시 인창동 244-1,경기도 구리시 건원대로 34번길 32-29,968816.570408409,1956190.17460319,127.146714,37.604604


In [163]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [164]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [165]:
#5 seoul university 
map

In [166]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_20.html")


In [167]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
# subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_20.csv", index = False)
